In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
#volvemos a cargar los dos dataframes
df_activity = pd.read_csv("files/customer_flight_activity.csv")
df_loyalty = pd.read_csv("files/customer_loyalty_history.csv")

## Eliminamos duplicados y columnas sin sentido

In [3]:
#lo primero que haremos será encargarnos de las filas duplicadas que teníamos en nuestro df de actividad
#estas filas contienen exactamente la misma información en cada una de sus columnas y sólo muestran información redundante. 
#Puesto que se trata de la misma persona (con el mismo número de identificación 'loyalty number'), podemos eliminarlas. Lo más probable es que sean errores cometidos al introducir los datos. 

df_activity.drop_duplicates (inplace = True)

In [4]:
#vamos a eliminar también la columna de country, que dijimos que contenía un único tipo de dato 'Canadá'. Esto es así porque se trata de una empresa canadiense y todos sus viajeros pertenecen a Canadá. 
df_loyalty.drop ('Country', axis=1, inplace= True)

In [5]:
#comprobamos que se han eliminado las filas duplicadas. En un principio teníamos 405.624 y hemos borrado 1864. 
print (f' nos quedan {df_activity.shape[0]} filas en nuestro df')

 nos quedan 403760 filas en nuestro df


## Unimos los dataframes

Ahora que hemos solucionado el problema de las filas y columnas con información redundante, vamos a unir los dos dataframes. 
Se trata de dos dataframes que contienen información sobre viajeros de diferentes aerolineas. El df_loyalty nos indica datos personales sobre el viajero (dirección, género, estado civil... ). El otro df contiene información sobre sus viajes realizados, reservados, puntos acumulados por vuelos, distancia recorrida, etc...
Nos interesa tener un único dataframe que contenga los datos de cada viajero con todos sus viajes. Por lo tanto, haremos la union mediant un left join. 

In [6]:
df = df_activity.merge( right=df_loyalty, how='left',
                        left_on='Loyalty Number', right_on='Loyalty Number')
df


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403755,999902,2018,12,0,0,0,0,0.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
403756,999911,2018,12,0,0,0,0,0.0,0,0,Newfoundland,St. John's,A1C 6H9,Male,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
403757,999940,2018,12,3,0,3,1233,123.0,0,0,Quebec,Quebec City,G1B 3L5,Female,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
403758,999982,2018,12,0,0,0,0,0.0,0,0,British Columbia,Victoria,V10 6T5,Male,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


## homogeneizamos el nombre de las columnas

In [7]:
# vamos a iterar por todas las columnas del DataFrame y a cada una de ellas la pondremos en minúsula y le aplicaremos un replace para unir las palabras mediante un guión. 
nuevas_columnas = {columna: columna.lower().replace(" ", "_") for columna in df.columns}

# comprobamos que hemos creado el diccionario correctamente
nuevas_columnas

{'Loyalty Number': 'loyalty_number',
 'Year': 'year',
 'Month': 'month',
 'Flights Booked': 'flights_booked',
 'Flights with Companions': 'flights_with_companions',
 'Total Flights': 'total_flights',
 'Distance': 'distance',
 'Points Accumulated': 'points_accumulated',
 'Points Redeemed': 'points_redeemed',
 'Dollar Cost Points Redeemed': 'dollar_cost_points_redeemed',
 'Province': 'province',
 'City': 'city',
 'Postal Code': 'postal_code',
 'Gender': 'gender',
 'Education': 'education',
 'Salary': 'salary',
 'Marital Status': 'marital_status',
 'Loyalty Card': 'loyalty_card',
 'CLV': 'clv',
 'Enrollment Type': 'enrollment_type',
 'Enrollment Year': 'enrollment_year',
 'Enrollment Month': 'enrollment_month',
 'Cancellation Year': 'cancellation_year',
 'Cancellation Month': 'cancellation_month'}

In [8]:
# una vez que tenemos el diccionario vamos a aplicar el método rename
df.rename(columns = nuevas_columnas, inplace = True)

# comprobamos que se han cambiado los nombres de las columnas
df.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


## homogeneizamos los valores de las columnas 

In [9]:
def homogeneizar(cadena):
    """
    Convierte una cadena para que todas sus letras estén en minúsulas y reemplaza espacios por guiones
    """
    return cadena.lower().replace(' ', '_')

In [10]:
lista_cambios = ['province', 'city', 'postal_code', 'gender', 'education', 'marital_status', 'loyalty_card', 'enrollment_type' ] 

In [11]:
# hemos hecho el cambio para una de las columnas, pero recordad que había dos columnas más que tenían este formato incorrecto
# vamos a crear una lista con los nombres de las columnas que queremos cambiar


# iteramos por la lista de columnas y a cada una de ellas le aplicamos la función que hemos creado con un '.apply()'
for col in lista_cambios:
    df[col] = df[col].apply(homogeneizar)

In [12]:
df.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,alberta,edmonton,t9g_1w3,female,bachelor,92552.0,married,aurora,7919.20,standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,ontario,toronto,m1r_4k3,male,college,NaN,single,nova,2887.74,standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,british_columbia,dawson_creek,u5i_4f1,female,college,NaN,divorced,nova,2838.07,standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,british_columbia,vancouver,v5r_1w3,male,bachelor,63253.0,married,star,4170.57,standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,ontario,toronto,p1l_8x8,female,bachelor,91163.0,divorced,star,6622.05,standard,2014,1,NaN,NaN


Ahora tenemos un dataframe limpio y homogéneo que podemos utilizar para realizar cualquier tipo de análisis. Lo guardamos para poder trabajar con él en los siguientes pasos utilizando pandas.

In [13]:
df.to_csv ('files/flights.csv')